In [86]:
import nltk
from nltk.corpus import stopwords
from string import punctuation
from os import listdir
import pandas as pd
from collections import Counter
import numpy as np
import string
nltk.downloader.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/zenger/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [221]:
import glob
folderName = 'AT&T'
files = listdir()
print(files)
df = pd.concat([pd.read_csv(f) for f in glob.glob('*.csv')], ignore_index = True)


['.DS_Store', 'Archive', 'bowSentiment.ipynb', 'AT&T7.csv', 'AT&T6.csv', '.ipynb_checkpoints', 'folderNameProcessed.csv']


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


In [212]:
def fix_csv(dataframe):
    pd.set_option('display.max_rows',20)
    pd.set_option('display.max_columns',10)
    dataframe['sv_score'] = 0.0
    dataframe = dataframe.dropna(subset = ['Text'])
    return dataframe

In [213]:
df = fix_csv(df)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [214]:
df

,Author,EndDate,Site,StartDate,Text,Title,sv_score
0,NaN,1052017,Reuters,12292016,WASHINGTON (Reuters) - The top U.S. intelligen...,\r\n U.S. spy chief 'resolute' ...,0.0
1,NaN,1052017,Reuters,12292016,"PALM BEACH, Fla./ MOSCOW (Reuters) - U.S. Pres...",\r\n Trump praises Putin for ho...,0.0
2,NaN,1052017,Reuters,12292016,ISTANBUL (Reuters) - A gunman opened fire on N...,\r\n Gunman kills 39 in Istanbu...,0.0
3,NaN,1052017,Reuters,12292016,WASHINGTON (Reuters) - The Republican-led U.S....,\r\n Rough start for U.S. Repub...,0.0
4,NaN,1052017,Reuters,12292016,NEW DELHI (Reuters) - Indian Prime Minister Na...,\r\n India's PM Modi defends ca...,0.0
5,NaN,1052017,Reuters,12292016,Ford CEO Mark Fields called the move “a vote o...,"\r\n Chided by Trump, Ford scra...",0.0
6,NaN,1052017,Reuters,12292016,The second largest U.S. automaker said it woul...,\r\n Ford scraps plan for $1.6 ...,0.0
7,NaN,1052017,Reuters,12292016,The peso MXN=D2 weakened 0.93 percent to 20.93...,\r\n Mexico's peso deepens loss...,0.0
8,NaN,1052017,Reuters,12292016,“Toyota Motor said will build a new plant in B...,\r\n Trump hits Toyota in lates...,0.0
9,NaN,1052017,Reuters,12292016,MEXICO CITY (Reuters) - Mexico’s central bank ...,\r\n Mexico central bank sells ...,0.0


In [215]:
def sentiment_Col(df):
    sid = SentimentIntensityAnalyzer()
    count = 0
    for index,row in df.iterrows():
        text = row['Text']
        stop_words = set(stopwords.words('english'))
        word_tokens = word_tokenize(text)
        filtered_sentence = [word for word in word_tokens if word.isalpha()]
        filtered_sentence = [w for w in word_tokens if not w in stop_words]
        filtered_sentence = [word for word in word_tokens if len(word)>1]
        final_sentence = []
        for w in filtered_sentence:
            final_sentence.append(w)
        final_sentence = TreebankWordDetokenizer().detokenize(final_sentence)
        cmp_Score = sid.polarity_scores(final_sentence)['compound']
        df.at[index,'sv_score'] = cmp_Score
        count +=1
    return df


In [216]:
sentiment_Col(df)

,Author,EndDate,Site,StartDate,Text,Title,sv_score
0,NaN,1052017,Reuters,12292016,WASHINGTON (Reuters) - The top U.S. intelligen...,\r\n U.S. spy chief 'resolute' ...,0.9959
1,NaN,1052017,Reuters,12292016,"PALM BEACH, Fla./ MOSCOW (Reuters) - U.S. Pres...",\r\n Trump praises Putin for ho...,0.9121
2,NaN,1052017,Reuters,12292016,ISTANBUL (Reuters) - A gunman opened fire on N...,\r\n Gunman kills 39 in Istanbu...,-0.9997
3,NaN,1052017,Reuters,12292016,WASHINGTON (Reuters) - The Republican-led U.S....,\r\n Rough start for U.S. Repub...,0.8908
4,NaN,1052017,Reuters,12292016,NEW DELHI (Reuters) - Indian Prime Minister Na...,\r\n India's PM Modi defends ca...,0.9977
5,NaN,1052017,Reuters,12292016,Ford CEO Mark Fields called the move “a vote o...,"\r\n Chided by Trump, Ford scra...",0.9918
6,NaN,1052017,Reuters,12292016,The second largest U.S. automaker said it woul...,\r\n Ford scraps plan for $1.6 ...,-0.5622
7,NaN,1052017,Reuters,12292016,The peso MXN=D2 weakened 0.93 percent to 20.93...,\r\n Mexico's peso deepens loss...,-0.8360
8,NaN,1052017,Reuters,12292016,“Toyota Motor said will build a new plant in B...,\r\n Trump hits Toyota in lates...,0.5525
9,NaN,1052017,Reuters,12292016,MEXICO CITY (Reuters) - Mexico’s central bank ...,\r\n Mexico central bank sells ...,-0.9505


In [222]:
df.to_csv('{}Processed.csv'.format(folderName))